In [1]:
import pandas as pd
import torch 

df=pd.read_csv("../../Data/data_wavenet/train_clean.csv")
df['signal']=df['signal']/df['signal'].max()

In [2]:
df['open_channels'].max()
df['open_channels'].min()
#nombre de samples
df.shape[0]

5000000

# Create time series dataset

In [3]:
class TimesereisDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.Series(), seqLen=10):
        super().__init__()
        self.df=df
        self.seqLen=seqLen
    
    def __len__(self):
        return self.df.shape[0]-self.seqLen-1
    
    def __getitem__(self,index):
        x=self.df.iloc[index:index+self.seqLen,1].values
        y=self.df.iloc[index+self.seqLen-1,2] 
        return x,y   
seqLen=50
timeSeriesDataset = TimesereisDataset(df,seqLen)

/tmp/ipykernel_184201/3899109925.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def __init__(self, df:pd.Series(), seqLen=10):


batchsize, batchnorm ? 

In [4]:
trainNumbers=int(len(timeSeriesDataset)*0.9)
trainDataset,testDataset=torch.utils.data.random_split(timeSeriesDataset,[trainNumbers,len(timeSeriesDataset)-trainNumbers])
trainDataLoader=torch.utils.data.DataLoader(trainDataset,batch_size=8,shuffle=True)
testDataLoader=torch.utils.data.DataLoader(testDataset,batch_size=8,shuffle=True)

In [6]:
trainNumbers


4499954

In [5]:

from Wavenet import WaveNet,WaveNetClassifier
from tqdm import tqdm 

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"
wavenetClassifierModel=WaveNetClassifier(seqLen,df['open_channels'].max()+1)
wavenetClassifierModel.to(device)

wavenetClassifierModel.train()

optimizer=torch.optim.AdamW(wavenetClassifierModel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
lossFunction = torch.nn.CrossEntropyLoss()

def calc_accuracy(Out,Y):
    max_vals, max_indices = torch.max(Out,1)
    train_acc = (max_indices == Y).sum().item()/max_indices.size()[0]
    return train_acc
  
epochs=1
globalStep=500

for epoch in range(epochs):
    for step, (x_train,y_train) in tqdm(enumerate(trainDataLoader),desc="Training"):
         x_train = torch.unsqueeze(x_train,dim=1).float()
         x_train.to(device)
         y_train.to(device)
         output=wavenetClassifierModel(x_train)
         output = torch.squeeze(output,dim=1)

         loss= lossFunction(output,y_train)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         if step%globalStep==0:
            # scheduler.step()
            # print(output.detach().numpy())
            # print(y_train.numpy())
            with torch.no_grad():
                accuracy=0
                loss=0
                for stepTest, (x_test,y_test) in tqdm(enumerate(testDataLoader),desc="Validation"):
                    x_test.to(device)
                    y_test.to(device)
                    x_test = torch.unsqueeze(x_test,dim=1).float()
                    output=wavenetClassifierModel(x_test)
                    output = torch.squeeze(output,dim=1)
                    accuracy+=calc_accuracy(output,y_test)*100
                    loss+= lossFunction(output,y_test).item()
                    if stepTest>200:
                        break
            print(f"loss for step {step} : {loss/stepTest}  :  Accuracy: {accuracy/stepTest} %")

         
    print(f"epch {epoch}")
         

Training: 0it [00:00, ?it/s]/home/hippo/anaconda3/lib/python3.9/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
Validation: 201it [00:05, 38.21it/s]
Training: 3it [00:06,  1.63s/it]

loss for step 0 : 2.418416332842699  :  Accuracy: 2.9228855721393034 %


Validation: 201it [00:04, 45.41it/s]
Training: 503it [00:44,  1.88it/s]

loss for step 500 : 2.289049127208653  :  Accuracy: 25.99502487562189 %


Validation: 201it [00:04, 46.53it/s]
Training: 1002it [01:21,  1.40it/s]

...

loss for step 3000 : 2.2970915445640907  :  Accuracy: 24.129353233830845 %


Validation: 201it [00:05, 36.27it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x7f70c75aaa50>
Traceback (most recent call last):
  File "/home/hippo/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 1193, in __iter__
    self.close()
  File "/home/hippo/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 1299, in close
    self.display(pos=0)
  File "/home/hippo/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 1492, in display
    self.sp(self.__str__() if msg is None else msg)
  File "/home/hippo/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 348, in print_status
    fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
  File "/home/hippo/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 342, in fp_write
    fp_flush()
  File "/home/hippo/anaconda3/lib/python3.9/site-packages/tqdm/utils.py", line 142, in inner
    return func(*args, **kwargs)
  File "/home/hippo/anaconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 4

loss for step 3500 : 2.284733992310899  :  Accuracy: 26.99004975124378 %


Training: 3890it [05:11, 12.50it/s]


KeyboardInterrupt: 